In [17]:
import numpy as np
import tensorflow as tf

imagePath = '/home/ppatnaik/Test_fruit/FRU_App_942.jpg'
modelFullPath = '/tmp/output_graph.pb'
labelsFullPath = '/tmp/output_labels.txt'


def create_graph():
    """Creates a graph from saved GraphDef file and returns a saver."""
    # Creates graph from saved graph_def.pb.
    with tf.gfile.FastGFile(modelFullPath, 'rb') as f:
        print(modelFullPath)
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
        _ = tf.import_graph_def(graph_def, name='')


def run_inference_on_image():
    answer = None

    if not tf.gfile.Exists(imagePath):
        tf.logging.fatal('File does not exist %s', imagePath)
        return answer

    image_data = tf.gfile.FastGFile(imagePath, 'rb').read()

    # Creates graph from saved GraphDef.
    create_graph()
   
    with tf.Session() as sess:
        
        softmax_tensor = sess.graph.get_tensor_by_name('final_result:0')
        predictions = sess.run(softmax_tensor,
                               {'DecodeJpeg/contents:0': image_data})
        print(predictions)
        predictions = np.squeeze(predictions)
        print(predictions)
        top_k = predictions.argsort()[-3:][::-1]  # Getting top 5 predictions
        print(top_k)
        f = open(labelsFullPath, 'rb')
        lines = f.readlines()
#         print(lines)
        labels = [str(w).replace("\n", "") for w in lines]
        for node_id in top_k:
            print(node_id)
            human_string = labels[node_id]
            score = predictions[node_id]
            print('%s (score = %.5f)' % (human_string, score))

        answer = labels[top_k[0]]
        return answer




In [18]:
if __name__ == '__main__':
    run_inference_on_image()

/tmp/output_graph.pb
[[ 0.06194948  0.92672336  0.01132712]]
[ 0.06194948  0.92672336  0.01132712]
[1 0 2]
1
b'apple\n' (score = 0.92672)
0
b'strawberry\n' (score = 0.06195)
2
b'orange\n' (score = 0.01133)
